<a href="https://colab.research.google.com/github/Fernando123Duran/Asistente_Composici-n_musical/blob/main/Notbooks/sprint_4/clasificador/Clasificador_partituras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/gdrive')
%cd '/gdrive/My Drive/Colab/proyecto_perfil/archivos_numpy/'

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/Colab/proyecto_perfil/archivos_numpy


In [ ]:
!ls

dataAumentation.npy  generos


In [5]:
import os
import numpy as np

class DatasetLoader:
  def __init__(self, path_dataset="../archivos_numpy/generos/"):
    self.path_dataset = path_dataset

  def shuffle(self, data, labels):
    data = np.array(data)
    labels = np.array(labels)
    p = np.random.permutation(len(data))
    return data[p], labels[p]

  def prepare_dataset(self, validation_split=0.2, shuffle=True, sequence_length=768):
    data=[]
    labels=[]
    label_count=0
    for genero in os.listdir(self.path_dataset):
      genero_data=np.load(self.path_dataset+genero, allow_pickle=True)
      # convert to dictionary
      genero_data=genero_data.item()
      for key in genero_data:
        data.append(genero_data[key])
        labels.append(label_count)
      label_count+=1
    # pad data
    for i in range(len(data)):
      if len(data[i])<sequence_length:
        # data[i]+=[0]*(sequence_length-len(data[i]))
        data[i]=np.pad(data[i], (0,sequence_length-len(data[i])), 'constant', constant_values=0)
      else:
        data[i]=data[i][:sequence_length]
    one_hot_labels=np.zeros((len(labels),label_count))
    for i in range(len(labels)):
      one_hot_labels[i][labels[i]]=1
    if shuffle:
      data,one_hot_labels=self.shuffle(data,one_hot_labels)
    split_index=int(len(data)*(1-validation_split))
    train_data=data[:split_index]
    train_labels=one_hot_labels[:split_index]
    validation_data=data[split_index:]
    validation_labels=one_hot_labels[split_index:]
    return train_data,train_labels,validation_data,validation_labels

  def read_for_evaluation(self, sequence_length=768):
    data=[]
    labels=[]
    label_count=0
    for genero in os.listdir(self.path_dataset):
      genero_data=np.load(self.path_dataset+genero, allow_pickle=True)
      # convert to dictionary
      genero_data=genero_data.item()
      for key in genero_data:
        data.append(genero_data[key])
        labels.append(label_count)
      label_count+=1
    # pad data
    for i in range(len(data)):
      if len(data[i])<sequence_length:
        data[i]+=[0]*(sequence_length-len(data[i]))
      else:
        data[i]=data[i][:sequence_length]
    one_hot_labels=np.zeros((len(labels),label_count))
    for i in range(len(labels)):
      one_hot_labels[i][labels[i]]=1
    return np.array(data),one_hot_labels

In [6]:
import tensorflow as tf

def build_model(output_length=10, sequence_length=768):
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(sequence_length,)),
        # convolutional layers
        tf.keras.layers.Reshape((sequence_length, 1)),
        tf.keras.layers.Conv1D(256, 12, activation='relu'),
        tf.keras.layers.Conv1D(128, 6, activation='relu'),
        # dense layers
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(output_length, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
!pip3 install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 24.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 21.6 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=54c3c761fb6e3a684b0e976930ae5ef9d728343ef96a74d8339ae195a8a75803
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import wandb
from wandb.keras import WandbMetricsLogger
wandb.init(project="clasificador2")

def train():
    dataset_loader = DatasetLoader("./generos/")
    train_data, train_labels, validation_data, validation_labels = dataset_loader.prepare_dataset()
    model = build_model(output_length=train_labels.shape[1], sequence_length=train_data.shape[1])
    model.fit(train_data, train_labels, epochs=50, validation_data=(validation_data, validation_labels),callbacks=[WandbMetricsLogger()])
    # Save model
    model.save("models/modelClasificador.h5")

In [ ]:
train()

Epoch 1/50
  4/190 [..............................] - ETA: 4s - loss: 26.3632 - accuracy: 0.1641  

190/190 [==============================] - 7s 18ms/step - loss: 1.9323 - accuracy: 0.5251 - val_loss: 0.8109 - val_accuracy: 0.7507
Epoch 2/50
190/190 [==============================] - 3s 17ms/step - loss: 0.3911 - accuracy: 0.8778 - val_loss: 0.1702 - val_accuracy: 0.9439
Epoch 3/50
190/190 [==============================] - 3s 17ms/step - loss: 0.0614 - accuracy: 0.9852 - val_loss: 0.0484 - val_accuracy: 0.9908
Epoch 4/50
190/190 [==============================] - 3s 17ms/step - loss: 0.0193 - accuracy: 0.9965 - val_loss: 0.0304 - val_accuracy: 0.9908
Epoch 5/50
190/190 [==============================] - 3s 17ms/step - loss: 0.0134 - accuracy: 0.9970 - val_loss: 0.0434 - val_accuracy: 0.9947
Epoch 6/50
190/190 [==============================] - 3s 17ms/step - loss: 0.0078 - accuracy: 0.9988 - val_loss: 0.0105 - val_accuracy: 0.9967
Epoch 7/50
190/190 [==============================] - 3s 17ms/step - loss: 0.0101 - accuracy: 0.9984 - val_loss: 0.0171 - val_accuracy: 0.9967
Epoch 8/50

In [ ]:
def cargarArchivosNPY(path):
    with open(path, 'rb') as f:
        datos = np.load(f,allow_pickle=True)
        datosCompletos=[]
        datos=datos.tolist()
        for da in datos:
          datosCompletos.append(np.array(datos[da]))
        print(len(datosCompletos[0]))
        print(datosCompletos[0])
        datosCompletos[0]=np.pad(datosCompletos[0], (0,200-len(datosCompletos[0])), 'constant', constant_values=0)
        print(len(datosCompletos[0]))
        print(datosCompletos[0])

In [ ]:
cargarArchivosNPY("./generos/dataAumentation_tobas.npy")

In [7]:
def calculate_inception_score(p_yx, eps=1E-16):
  # calculate p(y)
  p_y = np.expand_dims(p_yx.mean(axis=0), 0)
  # kl divergence for each image
  kl_d = p_yx * (np.log(p_yx + eps) - np.log(p_y + eps))
  # sum over classes
  sum_kl_d = kl_d.sum(axis=1)
  # average over images
  avg_kl_d = np.mean(sum_kl_d)
  # undo the logs
  is_score = np.exp(avg_kl_d)
  return is_score

In [37]:
import random
def leerDatosGeneros(path_dataset):
  data=[]
  seleccionCompleta=[]
  cantGen=0
  labels=[]
  for genero in os.listdir(path_dataset):
    genero_data=np.load(path_dataset+genero, allow_pickle=True)
    # convert to dictionary
    genero_data=genero_data.item()
    for key in genero_data:
      data.append(genero_data[key])
    random.shuffle(data) #mescla los datos
    for i in range(2):#numero de piezas por genero
      selecRa=random.randint(0,len(data))
      datoSelec=data[selecRa]
      tamanioParte=len(datoSelec)
      valIniRand=tamanioParte//2
      varRando=random.randint(valIniRand,tamanioParte)
      seleccionCompleta.append(datoSelec[:varRando])#Selecciona un tamanio random del array 
      labels.append(cantGen)
    data=[]
    cantGen+=1
    one_hot_labels=np.zeros((len(labels),cantGen))
    for i in range(len(labels)):
        one_hot_labels[i][labels[i]]=1
  return seleccionCompleta,cantGen,one_hot_labels



          
      

In [9]:
!pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 8.2 MB/s eta 0:00:00


In [38]:
#Leer modelo guardado
import onnxruntime as ort 
from scipy.special import softmax
from tqdm import tqdm
from tensorflow.python.keras.models import load_model
ort_session = ort.InferenceSession('../modelo/guardarModelo/Epoch200.onnx')


def generador_parte(parte):
  tam=768-len(parte)
  for i in tqdm(range(tam)):
    # print("parte : "+str(parte))
    part=np.array(parte)#dim(1,)
    part=np.expand_dims(part,axis=(0))#dime (1,n)
    part=np.int64(part)
    salida= ort_session.run(["output"],{"input":part})[0]
    salida=salida[:,-1,:]/1
    probs =softmax(salida,axis=-1)
    nota=np.argmax(probs,axis=-1)
    print("/////--> "+str(nota[0]))
    if nota[0]==2 or nota[0]==28:
      parte=np.append(parte,nota[0])
      break
    parte=np.append(parte,nota[0])
  return parte


def generador_completo(pathGeneros):
  partSelects,cantGen,one_hot_labels=leerDatosGeneros(pathGeneros)
  nvParts=[]
  for par in tqdm(partSelects):
    partGe=generador_parte(par)
    partGe=np.pad(partGe, (0,768-len(partGe)), 'constant', constant_values=0)
    nvParts.append(partGe)
  return nvParts,cantGen,one_hot_labels
  

def run_clasificacion():
  parteSelect,cantGen,one_hot_labels=generador_completo("./generos/")
  return parteSelect,cantGen,one_hot_labels
  

In [39]:
parteSelect,cantGen,one_hot_labels=run_clasificacion()

  0%|          | 1/673 [00:00<03:34,  3.13it/s]

/////--> 4



  0%|          | 2/673 [00:00<03:47,  2.96it/s]

/////--> 120



  0%|          | 3/673 [00:01<03:50,  2.90it/s]

...

/////--> 109



  5%|▌         | 34/645 [00:17<07:20,  1.39it/s]

/////--> 8


100%|██████████| 20/20 [08:42<00:00, 26.15s/it]

/////--> 28


In [40]:
parteSelect=np.array(parteSelect)
parteSelect.shape

(20, 768)

In [25]:
parteSelect[0].shape

(768,)

In [42]:
model = build_model(output_length=10, sequence_length=768)
# Load model
model.load_weights("models/modelClasificador.h5")
# get predictions
print("tamanio shape")
print(len(parteSelect))
predictions = model.predict(parteSelect, batch_size=len(parteSelect))
# get accuracy
print(predictions.shape)
accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(one_hot_labels, axis=1))
print("Accuracy: ", accuracy)
# print predictions converted to integers
print(np.argmax(predictions, axis=1))
# compute inception score
is_score = calculate_inception_score(predictions)
print("Inception score: ", is_score)

tamanio shape
20


1/1 [==============================] - 0s 209ms/step
(20, 10)
Accuracy:  0.95
[0 0 1 1 5 2 3 3 4 4 5 5 6 6 7 7 8 8 9 9]
Inception score:  9.0957985
